# KUx Colab End-to-End Notebook
This notebook provisions the KUx multimodal assistant on **Google Colab Pro+ (A100 80GB)**.
Follow the sections sequentially to install dependencies, prepare Retrieval-Augmented Generation (RAG) data, optionally fine-tune Qwen3-Omni, and launch the chatbot demo.

## 1. Check GPU runtime
Confirm the Colab session is running on an A100 80GB GPU.

In [ ]:
!nvidia-smi

## 2. Clone KUx and install dependencies
Set `REPO_URL` to your fork if you maintain a custom version. Editable installation exposes the `kux` package for the helper scripts.

In [ ]:
REPO_URL = 'https://github.com/themistymoon/KUx.git'
PROJECT_ROOT = '/content/KUx'

!git clone $REPO_URL
%cd $PROJECT_ROOT
!pip install -U pip
!pip install -r requirements.txt
!pip install -e .

### (Optional) Update Transformers nightly
Upgrade to the bleeding-edge Transformers build when Colab ships an older release than required by Qwen3-Omni's multimodal features.

In [ ]:
# !pip install -U "transformers@git+https://github.com/huggingface/transformers"

## 3. (Optional) Mount Google Drive
Persist FAISS indexes and LoRA adapters between sessions by mounting Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## 4. Authenticate with Hugging Face (if needed)
Login to access gated models or higher rate limits.

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

## 5. Configure storage locations
Set directories for RAG storage and optional LoRA adapters. Update these paths if you mounted Drive.

In [ ]:
from pathlib import Path

VECTOR_DB_DIR = Path('storage/vectorstore')
ADAPTER_DIR = Path('outputs/finetuned-qwen')
DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)
VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)
ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
print('Vector DB directory:', VECTOR_DB_DIR.resolve())
print('Adapter output directory:', ADAPTER_DIR.resolve())

## 6. Upload supervision dataset (optional)
Upload `train.jsonl` or other fine-tuning assets into the `data/` directory via Colab's file browser. Each line should contain either chat-style `{"messages": [...]}` entries or `{"instruction": ..., "response": ...}` pairs.

## 7. Crawl Kasetsart sources (optional)
Harvest approved KU pages to enrich the RAG corpus.

In [ ]:
CRAWL_OUTPUT = DATA_DIR / 'crawled'
CRAWL_OUTPUT.mkdir(exist_ok=True)

# Example crawl of the KU Computer Science site (adjust domains, depth, and page limits as needed).
# !python scripts/crawl_sites.py https://cs.sci.ku.ac.th --output $CRAWL_OUTPUT --max-depth 1 --max-pages 10

## 8. Build or update the FAISS vector store
Point the ingestion script at directories or files containing PDFs (≤20 pages recommended), CSVs, Markdown, or plain text.

In [ ]:
import shlex, subprocess
from pathlib import Path

PDF_DIRS = []  # e.g., [DATA_DIR / 'handbooks']
CSV_DIRS = []  # e.g., [DATA_DIR / 'datasets']
CRAWL_DIRS = [CRAWL_OUTPUT]  # include crawled text

def collect_existing(paths):
    collected = []
    for path in paths:
        p = Path(path)
        if p.exists():
            collected.append(p)
    return collected

normalized_paths = (
    collect_existing(PDF_DIRS) +
    collect_existing(CSV_DIRS) +
    collect_existing(CRAWL_DIRS)
)
if normalized_paths:
    cmd = ['python', 'scripts/build_vector_store.py', *map(str, normalized_paths), '--vector-db', str(VECTOR_DB_DIR)]
    print('Running:', ' '.join(shlex.quote(part) for part in cmd))
    subprocess.run(cmd, check=True)
else:
    print('⚠️  No existing document directories found. Upload PDFs/CSVs or enable crawling before ingesting.')


## 9. Fine-tune Qwen (optional)
Train LoRA adapters on your supervision dataset. Adjust hyperparameters with CLI flags or JSON configs if desired.

In [ ]:
import shlex, subprocess

DATASET_PATH = DATA_DIR / 'train.jsonl'
if DATASET_PATH.exists():
    cmd = [
        'python', 'scripts/train_qwen.py',
        '--dataset', str(DATASET_PATH),
        '--output-dir', str(ADAPTER_DIR)
    ]
    print('Running:', ' '.join(shlex.quote(part) for part in cmd))
    subprocess.run(cmd, check=True)
else:
    print('ℹ️  Skipping fine-tuning because data/train.jsonl is not present.')


## 10. Launch the KUx chatbot
Start the Gradio app with configurable model, adapter, and system prompt settings. The UI will display text, audio, image, and video inputs when a multimodal model is selected.

In [ ]:
import shlex, subprocess

MODEL_KEY = 'qwen3-omni-30b'  # or 'gpt-oss-120b' for text-only
SYSTEM_PROMPT = (
    'You are KUx, a Kasetsart University assistant. Provide verified guidance for the Computer Science programme.'
)

cmd = [
    'python', 'scripts/run_chatbot.py',
    '--model', MODEL_KEY,
    '--system-prompt', SYSTEM_PROMPT,
    '--vector-db', str(VECTOR_DB_DIR),
    '--share'
]
if ADAPTER_DIR.exists() and any(ADAPTER_DIR.iterdir()):
    cmd.extend(['--adapter', str(ADAPTER_DIR)])

print('Running:', ' '.join(shlex.quote(part) for part in cmd))
subprocess.run(cmd, check=False)

## 11. Shutdown (optional)
To stop the Gradio server, interrupt the cell above or run the following.

In [ ]:
# !pkill -f gradio